---
<div align="center">

# Connect-4 [Artificial Intelligence Project]
</div>

---

<div align="center">

## Search Problem 
</div>

A Search Problem is composed by multiple factors:
- States (Any possible configuration of the Board)
- Initial and Goal States
- Action Space (actions and their effects on the environment)
- Action Cost

---

In [1]:
import numpy as np
from copy import (deepcopy)
import random as rd
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Connect4_State:
    # Defining Board's Size
    NROWS = 6
    NCOLS = 7
    
    def __init__(self):
        # Matrix to Store the Board's Values
        self.board = np.zeros(shape=(self.NROWS, self.NCOLS), dtype=np.int8)

        # Array to keep track of the row's index for every column where a piece can be placed
        # Basically it's the Row idx for every column's height
        self.columns_height = np.full(shape=self.NCOLS, fill_value=(self.NROWS - 1), dtype=np.int8)

        # Defining the Possible Actions (Initially a piece can be placed in any column)
        self.actions = [col for col in range(self.NCOLS)]
        
        # Initializing a variable to track the current player
        self.current_player = 1
        
        # Variable to store the Winner (-1 - Game still running || 0 - Tie || 1 - PLayer 1 || 2 - Player 2 / AI)
        self.winner = -1

    def inside_board(self, x, y):
        # Checks if a position (x,y) exists inside the board's matrix
        return (x >= 0 and x < self.NROWS) and (y >= 0 and y < self.NCOLS)
    
    def move(self, ncol):
        # Creating a new state
        new_state = deepcopy(self)

        # Checks of the given column can be played on
        if(ncol in self.actions):
            # Inserting the move into the board
            new_state.board[new_state.columns_height[ncol]][ncol] = new_state.current_player
            
            # Updating the "ncol"'s height
            new_state.columns_height[ncol] -= 1
            
            # Checking if the column is full and therefore uncapable of receiving more pieces -> Changes the action space
            if (new_state.columns_height[ncol] < 0):
                new_state.actions.remove(ncol)

            # Updates the current Winner
            new_state.update_winner()

            # Updating current player for the next state
            new_state.current_player = 3 - new_state.current_player

        # Returns the New State (Could be the same as the previous if the action could not be performed)
        return new_state

    """ MIGHT NEED TO BE OPTIMIZED """
    def get_winner(self, currPiece, Directions:list[tuple]):
        for i in range(self.NROWS):
            for j in range(self.NCOLS):
                if (self.board[i][j] == currPiece):
                    for (x, y) in Directions:
                        row, col = (i, j)
                        winCondition = 1
                        while(winCondition < 4):
                            row += x
                            col += y
                            if (self.inside_board(row, col) and self.board[row][col] == currPiece):
                                winCondition += 1
                            else:
                                break
                        if (winCondition == 4):
                            self.winner = currPiece
                            return

    """ MIGHT NEED TO BE OPTIMIZED """
    def update_winner(self):
        Directions = [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]

        self.get_winner(1, Directions)
        if (self.winner == 1):
            return
            
        self.get_winner(2, Directions)
        if (self.winner == 2):
            return

        if (len(self.actions) == 0):
            self.winner = 0
    
    def is_over(self):
        # If the Winner corresponds to -1 then the game is not finished otherwise it is
        return self.winner != -1

    def __str__(self):
        # Converts the board into the style used in the Assignment 1 Paper
        DECODER = {0:'-', 1:'X', 2:'O'} 
        new_board = ""
        for x in range (self.NROWS):
            for y in range (self.NCOLS):
                if (y == 0):
                    new_board += DECODER[self.board[x][y]]
                else:
                    new_board += " " + DECODER[self.board[x][y]]
            new_board += '\n'
        return new_board
        # return str(self.board)

    def __hash__(self):
        return hash(str(self.board))

    def __eq__(self, other:object):
        if (not isinstance(other, Connect4)):
            return False
        return hash(self) == hash(other)
        

In [3]:
game = Connect4_State()
print(game)

- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -



In [4]:
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
print(game)

- O - - - - -
- X - - - - -
- O - - - - -
- X - - - - -
- O - - - - -
- X - - - - -



---
<div align="center">

## TreeNode Class
</div>

---

In [5]:
class TreeNode:
    # Screen Parameters
    SQSIZE = 80
    X_OFFSET = 60
    Y_OFFSET = 60
    
    # Circle - Screen Parameters
    CIRCLE_OFFSET = 10
    CIRCLE_POS = (X_OFFSET + SQSIZE//2, Y_OFFSET + SQSIZE//2)
    CIRCLE_RADIUS = (SQSIZE//2) - CIRCLE_OFFSET

    # RGB Colors
    BLACK = (0, 0, 0)
    WHITE = (255, 255, 255)
    BLUE = (135, 206, 250)
    DARK_BLUE = (0, 0, 255)
    RED = (255, 0, 0)
    YELLOW = (255, 255, 102)

    # Defining a Array with the Piece's Colors [Tuple: (BORDER, INNER CIRCLE)]
    PIECES_COLORS = [(BLACK, WHITE), # Empty Pieces
                     (BLACK, RED), # Player 1
                     (BLACK, YELLOW)] # Player 2
        
    def __init__(self, state, parent=None):
        # Stores a State of the Game
        self.state = state

        # Keeps a reference to his Parent Node
        self.parent = parent

        # Stores all the generated nodes
        self.children = []

    def generate_state(self, ncol):
        # Creates a new state after the move
        new_state = self.state.move(ncol)

        # Wraps it with a TreeNode 
        new_node = TreeNode(state=new_state, parent=self)

        # Inserts the New Node into the Current Node's Children
        self.children.append(new_node)
        
        # Returns the generated Node
        return new_node

    def draw_board(self, screen):
        # Draws Board's Shadow
        board_rect_shadow = pygame.Rect((self.X_OFFSET - 5, self.Y_OFFSET - 5), (self.SQSIZE*game.NCOLS + 2*5, self.SQSIZE*game.NROWS + 2*5))
        pygame.draw.rect(screen, self.BLACK, board_rect_shadow)

        # Draws Main Board
        board_rect = pygame.Rect((self.X_OFFSET, self.Y_OFFSET), (self.SQSIZE*game.NCOLS, self.SQSIZE*game.NROWS))
        pygame.draw.rect(screen, self.DARK_BLUE, board_rect)

        # Drawing Circles in the Board
        for row in range(self.state.NROWS):
            for col in range(self.state.NCOLS):
                # Getting the Colors from the Auxiliar List
                (Border_Color, Circle_Color) = self.PIECES_COLORS[self.state.board[row][col]]

                # Drawing the Circle's Border
                pygame.draw.circle(screen, Border_Color, (self.X_OFFSET + self.SQSIZE//2 + col*(self.SQSIZE),
                                                          self.Y_OFFSET + self.SQSIZE//2 + row*(self.SQSIZE)), self.CIRCLE_RADIUS)

                # Drawing the Main Circle
                pygame.draw.circle(screen, Circle_Color, (self.X_OFFSET + self.SQSIZE//2 + col*(self.SQSIZE),
                                                          self.Y_OFFSET + self.SQSIZE//2 + row*(self.SQSIZE)), int(0.9*self.CIRCLE_RADIUS))
                
    def __hash__(self):
        return hash(str(self.state, self.parent, [child for child in self.children]))
    
    def __eq__(self, other:object):
        if (not isinstance(other, TreeNode)):
            return False
        return self.__hash__() == other.__hash__()

    def __str__(self):
        return str(self.state)
        

In [6]:
initial_state = Connect4_State()
root = TreeNode(initial_state)

---
<div align="center">

## Algorithms
</div>

---

### A* Search

In [7]:
def A_Star_Search():
    """ A* Search """
    pass

### MinMax

In [8]:
def MinMax():
    """ MinMax with Alpha-Beta Prunning - EXTRA """
    pass

### Monte Carlo Tree Search

In [9]:
def MCTS():
    """ Monte Carlo Tree Search """
    pass

---
<div align="center">

## Graphical User Interface
</div>

---

In [10]:
class APP:
    def __init__(self, player1="Human", player2="Human"):
        self.root = TreeNode(state=Connect4_State())
        self.current_node = deepcopy(self.root)
        
        self.WIDTH = self.root.state.NCOLS*self.root.SQSIZE + 2*self.root.X_OFFSET
        self.HEIGHT = self.root.state.NROWS*self.root.SQSIZE + 2*self.root.Y_OFFSET

        self.player1 = player1
        self.player2 = player2
        
        self.clicked = False
        self.menu = "Main"

    def get_current_player_action(self):
        if (self.current_node.state.current_player == 1):
            return self.player1
        else:
            return self.player2
    
    """ TERMINAL METHODS """
    
    def run_terminal(self):
        # Creating the Game Loop inside the Terminal (It Runs while we haven't reached a Final State)
        while not self.current_node.state.is_over():

            # Printing current board configuration
            print(f"\nCURRENT BOARD: \n{self.current_node.state}")

            # Getting the Current Action to Perform (if it's a algorithm or a player)
            current_action = self.get_current_player_action()
            
            if (current_action == "Human"):
                # Requesting a column to play at
                ncol = int(input(f"| Player {self.current_node.state.current_player} | Choose a Column to Play: "))
            
            elif(current_action == "Random"):
                # Randomizing a col to play at & printing wich one it was
                ncol = rd.randrange(0, len(self.current_node.state.actions))
                print(f"| Randomizer AI | Choosed to Play in the {ncol}th column ")
            
            # Creating a new Node by making a move into the "ncol" column
            new_node = self.current_node.generate_state(ncol)

            # Updating Current Node
            self.current_node = new_node

        # Printing Final Board Configuration
        print(f"\nFINAL BOARD: \n{self.current_node.state}")
        
        # Checking if it was a Tie
        if self.current_node.state.winner == 0:
            print("Tie")
        else: # A Player Won
            print(f"-> Player {self.current_node.state.winner} Wins!")


    """ GUI METHODS """
    
    def write(self, font, text, size, color, bg_color, bold, pos, screen):
        # Writes Text into the Screen
        letra = pygame.font.SysFont(font, size, bold)
        frase = letra.render(text, 1, color, bg_color)
        screen.blit(frase, pos)
    
    def draw(self, screen):
        # Filling the Background with Blue
        screen.fill(self.current_node.BLUE)

        # Drawing the Current Board Elements
        self.current_node.draw_board(screen)
    
    def run_gui(self):
        # Initializing Window / Screen
        screen = pygame.display.set_mode((self.WIDTH, self.HEIGHT))
        pygame.display.set_caption("Connect-4")
        ICON_IMG = pygame.image.load('./Assets/Connect-Four.png').convert_alpha()
        pygame.display.set_icon(ICON_IMG)

        # Create a Flag to keep track of current state of the Application / GUI
        run = True

        # Main Loop
        while run:
            
            # Draws the Game Elements into the Screen
            self.draw(screen)

            # If we haven't reached a Final State then keep playing
            if not self.current_node.state.is_over():

                # Getting current player action
                current_action = self.get_current_player_action()

                # Checking if the Player is Human
                if (current_action == "Human"):
                    
                    # Checking if we pressed the mouse 1 button and therefore changed the self.clicked flag
                    if not self.clicked and pygame.mouse.get_pressed()[0] == 1:
                        
                        # Getting Mouse Position
                        (y, x) = pygame.mouse.get_pos()
    
                        # Modifying the Mouse Coordinates to match with the Screen Stylling
                        x = (x - self.current_node.X_OFFSET) // self.current_node.SQSIZE
                        y = (y - self.current_node.Y_OFFSET) // self.current_node.SQSIZE
    
                        # Checking if the coordenates exist in the board. If so, add a piece to the column that the mouse was pressed on
                        if (self.current_node.state.inside_board(x, y)):
                            self.current_node = self.current_node.generate_state(y)
    
                        # Updating the "clicked" flag
                        self.clicked = True
        
                    # Checking if we released the mouse 1 button and therefore changed the self.clicked flag
                    if self.clicked and pygame.mouse.get_pressed()[0] == 0:
                        
                        # Updating the "clicked" flag
                        self.clicked = False

                # Checking if the Player is set to Random
                elif(current_action == "Random"):
                    # Randomizing a col to play at
                    ncol = rd.randrange(0, len(self.current_node.state.actions))

                    # Executing the Move
                    self.current_node = self.current_node.generate_state(ncol)
            
            # Main Event Loop
            for event in pygame.event.get():
                if (event.type == pygame.QUIT):
                    run = False

            # Updates the Window
            pygame.display.update()
        pygame.quit()

In [11]:
App = APP(player1="Human", player2="Random")

In [12]:
# App.run_terminal()

In [13]:
App.run_gui()